# Init Libraries

In [ ]:
import pandas as pd
import streamlit as st
import pickle


# Load Data, Preprocess Movies

In [ ]:
movies = pd.read_csv("csv/movies.csv")
links = pd.read_csv("csv/links.csv")
ratings = pd.read_csv("csv/ratings.csv")
movies = movies.merge(links,on="movieId",how="left")
movies["year"] = movies["title"].str[-5:-1]
movies = movies.drop(["imdbId"],axis=1).copy()
movies = movies.drop_duplicates().copy()
movies = movies.loc[movies["title"] != ""]
movies = movies.sort_values(["title","year"]).copy()

# Aggregate Ratings by movieId

In [ ]:
ratings.groupby(["movieId"], as_index=False).agg({"rating":"sum"}).describe()

In [ ]:
aggRatings = ratings.groupby('movieId').agg(Rating_Count=('rating', 'count'),
                                            Rating_Mean=('rating', 'mean')).reset_index()
aggRatings = aggRatings.sort_values("Rating_Count",ascending=False)

# Merge Ratings to Movies

In [ ]:
movies = movies.merge(aggRatings,on="movieId",how="left").copy()

# Pickle Movielist

In [ ]:
fileObj = open('pickle/movielist.sav', 'wb')
pickle.dump(movies[["movieId","title","genres","year","tmdbId"]],fileObj)
fileObj.close()

# Build Genre-List

In [45]:
genres = movies['genres'].str.split('|')
genres = genres.explode('genres')
genrelist = genres.unique()
# genrelist = pd.DataFrame(genrelist).tolist()

In [46]:
#genrelist

array(['Action', 'Drama', 'Thriller', 'War', 'Adventure', 'Comedy',
       'Documentary', 'Fantasy', 'Musical', 'Horror', 'Mystery',
       'Romance', 'Children', 'Sci-Fi', 'Crime', 'Animation', 'Film-Noir',
       'IMAX', 'Western', '(no genres listed)'], dtype=object)

In [ ]:
#movies["genres"].sort_values()

# and Pickle it as a list

In [ ]:
#genrelist = genres[0].tolist()

In [47]:
fileObj = open('pickle/genres.sav', 'wb')
pickle.dump(genrelist,fileObj)
fileObj.close()

# Check pickles

In [48]:
movielist = pickle.load(open('pickle/movielist.sav', 'rb'))
movielist

,movieId,title,genres,year,tmdbId
0,117867,'71 (2014),Action|Drama|Thriller|War,2014,252178.0
1,97757,'Hellboy': The Seeds of Creation (2004),Action|Adventure|Comedy|Documentary|Fantasy,2004,72867.0
2,26564,'Round Midnight (1986),Drama|Musical,1986,14670.0
3,27751,'Salem's Lot (2004),Drama|Horror|Mystery|Thriller,2004,36763.0
4,779,'Til There Was You (1997),Drama|Romance,1997,32872.0
...,...,...,...,...,...
9737,2600,eXistenZ (1999),Action|Sci-Fi|Thriller,1999,1946.0
9738,5507,xXx (2002),Action|Crime|Thriller,2002,7451.0
9739,33158,xXx: State of the Union (2005),Action|Crime|Thriller,2005,11679.0
9740,2478,¡Three Amigos! (1986),Comedy|Western,1986,8388.0


In [49]:
genres = pickle.load(open('pickle/genres.sav', 'rb'))
genres

array(['Action', 'Drama', 'Thriller', 'War', 'Adventure', 'Comedy',
       'Documentary', 'Fantasy', 'Musical', 'Horror', 'Mystery',
       'Romance', 'Children', 'Sci-Fi', 'Crime', 'Animation', 'Film-Noir',
       'IMAX', 'Western', '(no genres listed)'], dtype=object)